In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df=pd.read_csv('C:\\Users\\leman\\Documents\\makine ögrenmesi atolyesi\\Parkison_Dataset.csv')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score

# Örnek bir veri seti oluşturma 
# Özellikler (X) ve hedef değişken (y) olarak ayırma
X = df.drop('class', axis=1)  
y = df['class']

# Veriyi eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Veriyi ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelleri oluşturma ve eğitme
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_scaled, y_train)

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

nn_model = MLPClassifier(max_iter=1000, random_state=42)
nn_model.fit(X_train_scaled, y_train)

svm_model = SVC(probability=True)
svm_model.fit(X_train_scaled, y_train)

# Topluluk modelini oluşturma ve eğitme
voting_model = VotingClassifier(
    estimators=[
        ('logistic', logistic_model),
        ('knn', knn_model),
        ('nn', nn_model),
        ('svm', svm_model)
    ],
    voting='soft'  # 'hard' çoğunluk oylaması anlamına gelir, 'soft' olasılıkların ortalaması anlamına gelir
)
voting_model.fit(X_train_scaled, y_train)

# Tahminler
y_pred_logistic = logistic_model.predict(X_test_scaled)
y_pred_knn = knn_model.predict(X_test_scaled)
y_pred_nn = nn_model.predict(X_test_scaled)
y_pred_svm = svm_model.predict(X_test_scaled)
y_pred_voting = voting_model.predict(X_test_scaled)

# Performans metriklerinin hesaplanması
def print_metrics(name, y_true, y_pred):
    print(f"{name}:")
    print("  Doğruluk (Accuracy):", accuracy_score(y_true, y_pred))
    print("  F1 Skoru:", f1_score(y_true, y_pred, average='weighted'))
    print("  Geri Çağırma Skoru (Recall):", recall_score(y_true, y_pred, average='weighted'))
    print()

# Her bir modelin performansını yazdırma
print_metrics("Lojistik Regresyon", y_test, y_pred_logistic)
print_metrics("KNN", y_test, y_pred_knn)
print_metrics("Yapay Sinir Ağı", y_test, y_pred_nn)
print_metrics("SVM", y_test, y_pred_svm)
print_metrics("Voting Topluluk Modeli", y_test, y_pred_voting)
# topluluk öğrenme yöntemlerinden voting yöntemi istediğimiz başarıyı göstermedi

Lojistik Regresyon:
  Doğruluk (Accuracy): 0.868421052631579
  F1 Skoru: 0.8644688644688646
  Geri Çağırma Skoru (Recall): 0.868421052631579

KNN:
  Doğruluk (Accuracy): 0.8881578947368421
  F1 Skoru: 0.8759252853683847
  Geri Çağırma Skoru (Recall): 0.8881578947368421

Yapay Sinir Ağı:
  Doğruluk (Accuracy): 0.8947368421052632
  F1 Skoru: 0.8915750915750916
  Geri Çağırma Skoru (Recall): 0.8947368421052632

SVM:
  Doğruluk (Accuracy): 0.8552631578947368
  F1 Skoru: 0.8352534562211981
  Geri Çağırma Skoru (Recall): 0.8552631578947368

Voting Topluluk Modeli:
  Doğruluk (Accuracy): 0.875
  F1 Skoru: 0.8689778953334593
  Geri Çağırma Skoru (Recall): 0.875



In [3]:
from sklearn.ensemble import BaggingClassifier

# Özellikler (X) ve hedef (y) ayrıştırma
X = df.drop('class', axis=1)
y = df['class']

# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Özellikleri normalize etme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Yapay Sinir Ağı (ANN) modeli
ann_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)

# KNN modeli
knn_model = KNeighborsClassifier()

# SVM modeli
svm_model = SVC(probability=True, random_state=42)

# Bagging uygulaması
bagging_ann = BaggingClassifier(base_estimator=ann_model, n_estimators=10, random_state=42)
bagging_knn = BaggingClassifier(base_estimator=knn_model, n_estimators=10, random_state=42)
bagging_svm = BaggingClassifier(base_estimator=svm_model, n_estimators=10, random_state=42)

# Modelleri eğitme
bagging_ann.fit(X_train_scaled, y_train)
bagging_knn.fit(X_train_scaled, y_train)
bagging_svm.fit(X_train_scaled, y_train)

# Test seti üzerinde tahmin yapma
y_pred_ann = bagging_ann.predict(X_test_scaled)
y_pred_knn = bagging_knn.predict(X_test_scaled)
y_pred_svm = bagging_svm.predict(X_test_scaled)

# Doğruluk skorlarını hesaplama
accuracy_ann = accuracy_score(y_test, y_pred_ann)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print(f'ANN Bagging Doğruluk Skoru: {accuracy_ann:.2f}')
print(f'KNN Bagging Doğruluk Skoru: {accuracy_knn:.2f}')
print(f'SVM Bagging Doğruluk Skoru: {accuracy_svm:.2f}')

#bagging yöntemi voting yöntemine göre daha başarılı karar verdi

ANN Bagging Doğruluk Skoru: 0.89
KNN Bagging Doğruluk Skoru: 0.89
SVM Bagging Doğruluk Skoru: 0.89
